In this tutorial, I will show how to transform documents from one vector representation into another. This process serves two goals:

1. To bring out hidden structure in the corpus, discover relationships between words and use them to describe the documents in a new and (hopefully) more semantic way.   
2. To make the document representation more compact. This both improves efficiency (new representation consumes less resources) and efficacy (marginal data trends are ignored, noise-reduction).

In [11]:
import logging
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities

In [13]:
if (os.path.exists("/tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('/tmp/deerwester.dict')
    corpus = corpora.MmCorpus('/tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

2017-02-06 14:50:57,725 : INFO : loading Dictionary object from /tmp/deerwester.dict
2017-02-06 14:50:57,726 : INFO : loaded /tmp/deerwester.dict
2017-02-06 14:50:57,728 : INFO : loaded corpus index from /tmp/deerwester.mm.index
2017-02-06 14:50:57,729 : INFO : initializing corpus reader from /tmp/deerwester.mm
2017-02-06 14:50:57,730 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


Used files generated from first tutorial


In [19]:
print("dictionary:")
print(dictionary.token2id)
print("\ncorpus:")
for c in corpus:
    print(c)

dictionary:
{'user': 4, 'minors': 11, 'survey': 7, 'eps': 8, 'computer': 2, 'interface': 1, 'response': 3, 'graph': 10, 'system': 5, 'human': 0, 'time': 6, 'trees': 9}

corpus:
[(0, 1.0), (1, 1.0), (2, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0)]
[(1, 1.0), (4, 1.0), (5, 1.0), (8, 1.0)]
[(0, 1.0), (5, 2.0), (8, 1.0)]
[(3, 1.0), (4, 1.0), (6, 1.0)]
[(9, 1.0)]
[(9, 1.0), (10, 1.0)]
[(9, 1.0), (10, 1.0), (11, 1.0)]
[(7, 1.0), (10, 1.0), (11, 1.0)]


tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in information retrieval and text mining.   
The tf-idf value increases proportionally to the number of times a word appears in the document, but is offset by the frequency of the word in the corpus, which helps to adjust for the fact that some words appear more frequently in general.

In [14]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2017-02-06 14:55:24,083 : INFO : collecting document frequencies
2017-02-06 14:55:24,085 : INFO : PROGRESS: processing document #0
2017-02-06 14:55:24,086 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [21]:
corpus_tfidf = tfidf[corpus] # step 2 -- use the model to transform corpus
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(2, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.3244870206138555), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.44424552527467476)]
[(1, 0.5710059809418182), (4, 0.4170757362022777), (5, 0.4170757362022777), (8, 0.5710059809418182)]
[(0, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.45889394536615247), (6, 0.6282580468670046)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(7, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


Latent semantic indexing (LSI) is an indexing and retrieval method that uses a mathematical technique called (SVD) to identify patterns in the relationships between the terms and concepts contained in an unstructured collection of text. LSI is based on the principle that words that are used in the same contexts tend to have similar meanings

In [22]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2017-02-06 15:47:02,363 : INFO : using serial LSI version on this node
2017-02-06 15:47:02,366 : INFO : updating model with new documents
2017-02-06 15:47:02,368 : INFO : preparing a new chunk of documents
2017-02-06 15:47:02,375 : INFO : using 100 extra samples and 2 power iterations
2017-02-06 15:47:02,377 : INFO : 1st phase: constructing (12, 102) action matrix
2017-02-06 15:47:02,382 : INFO : orthonormalizing (12, 102) action matrix
2017-02-06 15:47:02,412 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2017-02-06 15:47:02,435 : INFO : computing the final decomposition
2017-02-06 15:47:02,437 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2017-02-06 15:47:02,442 : INFO : processed documents up to #9
2017-02-06 15:47:02,452 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-02-06 15:47:02,453 : INFO : topic #

In [23]:
#lsi.print_topics(2)

2017-02-06 15:47:52,511 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-02-06 15:47:52,513 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [28]:
print("correlation of the corpus with the two topics:")
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

correlation of the corpus with the two topics:
[(0, 0.066007833960904427), (1, -0.52007033063618513)]
[(0, 0.19667592859142619), (1, -0.76095631677000453)]
[(0, 0.089926399724465589), (1, -0.72418606267525076)]
[(0, 0.075858476521782722), (1, -0.63205515860034278)]
[(0, 0.10150299184980197), (1, -0.57373084830029519)]
[(0, 0.70321089393783154), (1, 0.16115180214025912)]
[(0, 0.87747876731198349), (1, 0.16758906864659551)]
[(0, 0.90986246868185794), (1, 0.14086553628719162)]
[(0, 0.61658253505692828), (1, -0.053929075663892739)]


In [27]:
lsi.save('/tmp/model.lsi') # same for tfidf, lda, ...
lsi = models.LsiModel.load('/tmp/model.lsi')

2017-02-06 16:11:51,072 : INFO : saving Projection object under /tmp/model.lsi.projection, separately None
2017-02-06 16:11:51,078 : INFO : saved /tmp/model.lsi.projection
2017-02-06 16:11:51,079 : INFO : saving LsiModel object under /tmp/model.lsi, separately None
2017-02-06 16:11:51,080 : INFO : not storing attribute dispatcher
2017-02-06 16:11:51,081 : INFO : not storing attribute projection
2017-02-06 16:11:51,083 : INFO : saved /tmp/model.lsi
2017-02-06 16:11:51,084 : INFO : loading LsiModel object from /tmp/model.lsi
2017-02-06 16:11:51,086 : INFO : loading id2word recursively from /tmp/model.lsi.id2word.* with mmap=None
2017-02-06 16:11:51,087 : INFO : setting ignored attribute dispatcher to None
2017-02-06 16:11:51,088 : INFO : setting ignored attribute projection to None
2017-02-06 16:11:51,089 : INFO : loaded /tmp/model.lsi
2017-02-06 16:11:51,090 : INFO : loading LsiModel object from /tmp/model.lsi.projection
2017-02-06 16:11:51,091 : INFO : loaded /tmp/model.lsi.projection
